### Connecting Jupyter Notebook to database file

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

### Listing all tables in the database

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Summary data for every playlist 

In [36]:
%%sql
WITH playlist_info AS
(
    select 
        p.playlist_id,
        p.name playlist_name,
        t.name track_name,
        CAST(t.milliseconds/1000 AS FLOAT) track_length
    FROM playlist p 
    LEFT JOIN playlist_track pt ON pt.playlist_id=p.playlist_id
    LEFT JOIN track t ON t.track_id=pt.track_id
)
SELECT 
    playlist_id,
    playlist_name,
    COUNT(track_name) number_of_tracks,
    SUM(track_length) length_seconds
FROM playlist_info 
GROUP BY 1
ORDER BY 1
LIMIT 5;

Done.


playlist_id,playlist_name,number_of_tracks,length_seconds
1,Music,3290,876049.0
2,Movies,0,None
3,TV Shows,213,500987.0
4,Audiobooks,0,None
5,90’s Music,1477,397970.0


### Summary data on tracks from different albums
- "Jagged Little Pill"

In [35]:
%%sql 
WITH track_info AS
    (                
     SELECT
         t.name,
         ar.name artist,
         al.title album_name,
         mt.name media_type,
         g.name genre,
         t.milliseconds length_milliseconds
     FROM track t
     INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
     INNER JOIN genre g ON g.genre_id = t.genre_id
     INNER JOIN album al ON al.album_id = t.album_id
     INNER JOIN artist ar ON ar.artist_id = al.artist_id
    )

SELECT * FROM track_info
WHERE album_name = "Jagged Little Pill"
LIMIT 5;

Done.


name,artist,album_name,media_type,genre,length_milliseconds
All I Really Want,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,284891
You Oughta Know,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,249234
Perfect,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,188133
Hand In My Pocket,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,221570
Right Through You,Alanis Morissette,Jagged Little Pill,MPEG audio file,Rock,176117


### Customers that spent more than 90$ in the store

In [29]:
%%sql
CREATE VIEW customer_gt_90_dollars AS
    SELECT c.* 
    FROM customer c
    INNER JOIN invoice i on i.customer_id=c.customer_id
    GROUP BY 1
    HAVING SUM(i.total)>90;

Done.


[]

In [34]:
%%sql
SELECT *
FROM customer_gt_90_dollars
LIMIT 5;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4


### Customers from USA

In [26]:
%%sql
CREATE VIEW customer_usa AS
    SELECT 
        c.* 
    FROM 
        customer c
    INNER JOIN 
        invoice i on i.customer_id=c.customer_id
    WHERE 
        c.country=="USA";

Done.


[]

In [33]:
%%sql
SELECT *
FROM customer_usa
LIMIT 5;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4


### Customers in USA who spend more than 90$ in the store

In [39]:
%%sql
SELECT * from customer_usa
UNION
SELECT * from customer_gt_90_dollars
LIMIT 5;

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4


### Identifying number of top customers sales, that the support agents have served.

In [41]:
%%sql
CREATE VIEW customers_usa_gt_90_view AS 
    SELECT 
        customer_id,
        support_rep_id employee_id
    FROM customer_usa
    
    INTERSECT
    
    SELECT 
        customer_id,
        support_rep_id employee_id
    FROM customer_gt_90_dollars;

Done.


[]

In [42]:
%%sql
SELECT 
    e.first_name || " " || e.last_name employee_name,
    COUNT(c.customer_id) customers_usa_gt_90
FROM employee e
LEFT JOIN customers_usa_gt_90_view c on e.employee_id=c.employee_id
WHERE e.title = "Sales Support Agent"
GROUP BY e.employee_id
ORDER BY 1

Done.


employee_name,customers_usa_gt_90
Jane Peacock,0
Margaret Park,2
Steve Johnson,2


### Sales data on customers from India

In [44]:
%%sql
WITH 
    India as(
        SELECT 
            customer_id
        FROM customer
        WHERE country="India"
    ),
    sum_total as(
        SELECT 
            c.customer_id,
            c.first_name || " " || c.last_name customer_name,
            SUM(i.total) total_purchases
        FROM customer c
        INNER JOIN invoice i ON i.customer_id=c.customer_id
        GROUP BY 1
    ),
    main as(
        SELECT 
            customer_name,
            round(total_purchases,2)
        FROM India i
        INNER JOIN sum_total s ON s.customer_id=i.customer_id
    )
SELECT * FROM main
ORDER BY customer_name
        
        

Done.


customer_name,"round(total_purchases,2)"
Manoj Pareek,111.87
Puja Srivastava,71.28


### Finding the customer from each country that has spent the most money at store

In [47]:
%%sql
WITH
    customer_country_purchases AS
        (
         SELECT
             i.customer_id,
             c.country,
             SUM(i.total) total_purchases
         FROM invoice i
         INNER JOIN customer c ON i.customer_id = c.customer_id
         GROUP BY 1, 2
        ),
    country_max_purchase AS
        (
         SELECT
             country,
             MAX(total_purchases) max_purchase
         FROM customer_country_purchases
         GROUP BY 1
        ),
    country_best_customer AS
        (
         SELECT
            cmp.country,
            cmp.max_purchase,
            (
             SELECT ccp.customer_id
             FROM customer_country_purchases ccp
             WHERE ccp.country = cmp.country AND cmp.max_purchase = ccp.total_purchases
            ) customer_id
         FROM country_max_purchase cmp
        )
SELECT
    cbc.country country,
    c.first_name || " " || c.last_name customer_name,
    ROUND(cbc.max_purchase,2) total_purchased
FROM customer c
INNER JOIN country_best_customer cbc ON cbc.customer_id = c.customer_id
ORDER BY 1 ASC

Done.


country,customer_name,total_purchased
Argentina,Diego Gutiérrez,39.6
Australia,Mark Taylor,81.18
Austria,Astrid Gruber,69.3
Belgium,Daan Peeters,60.39
Brazil,Luís Gonçalves,108.9
Canada,François Tremblay,99.99
Chile,Luis Rojas,97.02
Czech Republic,František Wichterlová,144.54
Denmark,Kara Nielsen,37.62
Finland,Terhi Hämäläinen,79.2


### Identifying best selling genre in USA

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

Artist Name | Genre
 --- | ---
Regal | Hip-Hop
Red Tone | Punk
Meteor and the Girls | Pop
Slim Jim Bites | Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [57]:
%%sql
WITH
    genre_purchases AS
        (
         SELECT
             t.track_id,
             g.name genre,
             SUM(i.total) total_purchases
         FROM genre g
         INNER JOIN track t ON t.genre_id = g.genre_id
         INNER JOIN invoice_line il ON il.track_id = t.track_id
         INNER JOIN invoice i ON i.invoice_id=il.invoice_id
         GROUP BY 2,1
        )
    
SELECT
    genre,
    ROUND(SUM(total_purchases),2) total_sales
FROM genre_purchases 
GROUP BY 1
ORDER BY 1 ASC

Done.


genre,total_sales
Alternative,1095.93
Alternative & Punk,4841.1
Blues,1379.07
Classical,361.35
Drama,5.94
Easy Listening,951.39
Electronica/Dance,614.79
Heavy Metal,70.29
Hip Hop/Rap,463.32
Jazz,1302.84


From the analysis Alternative seems to be doing the best in terms of sales so **Taylor Swift** a.k.a "The music industry" must be worshipped and worshipped regardless ;)  
But from the list of candidates the best choice would be **Red Tone** because they are Punk and, Alternative and Punk is the second best performing genre.

### Gauging Employee performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

For which we identify number of top customers sales, that the employees as support agents have served.

In [61]:
%%sql
SELECT 
    e.first_name || " " || e.last_name employee_name,
    COUNT(c.customer_id) no_top_customers
FROM employee e
LEFT JOIN customer_gt_90_dollars c on e.employee_id=c.support_rep_id
WHERE e.title = "Sales Support Agent"
GROUP BY e.employee_id
ORDER BY 1

Done.


employee_name,no_top_customers
Jane Peacock,8
Margaret Park,5
Steve Johnson,5


## To-do
1. Analyze the sales data for customers from each different country. You have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.   
You have been directed to calculate data, for each country, on the:
    * total number of customers
    * total value of sales
    * average value of sales per customer
    * average order value
2. Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.  
We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.